In [3]:
from dependencies import *
import sys
sys.path.append("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/Scripts/Functions")

#For precipitation mask

In [9]:
hr_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/RhiresD_1971_2022.nc"
mask_output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_mask.nc"

ds_hr = xr.open_dataset(hr_path)

varname = "RhiresD" 

mask_rhiresd = xr.where(~np.isnan(ds_hr[varname].isel(time=0)), 1, 0)

mask_rhiresd = mask_rhiresd.squeeze(drop=True)

mask_rhiresd.to_netcdf(mask_output_path)

print(f"Mask saved at {mask_output_path}")


Mask saved at /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_mask.nc


#For temperature mask!

In [10]:
hr_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/TabsD_1971_2022.nc"
mask_output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_mask.nc"

ds_hr = xr.open_dataset(hr_path)

varname = "TabsD" 

mask_tabsd = xr.where(~np.isnan(ds_hr[varname].isel(time=0)), 1, 0)

mask_tabsd = mask_tabsd.squeeze(drop=True)



In [ ]:
mask_tabsd.to_netcdf(mask_output_path)

print(f"Mask saved at {mask_output_path}")

#Application of the mask to the two datasets : RhiresD and TabsD masks (1km) to the bicubically interpolated 1 km datasets 

In [ ]:
bicubic_ds = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_1971_2022_bicubic.nc",chunks={"time": 100})

# Masking
masked_bicubic_rhiresd = bicubic_ds * mask_rhiresd

masked_bicubic_rhiresd.to_netcdf("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_1km_bicubic_Swiss_features_masked.nc",
                                 engine="netcdf4",
                                    compute=True)


Lazy loading was used below to allow for parallel computation. The data was not loading due to xarray loading everything at once and the kernel was crashing

In [ ]:
# Load with chunks (lazy loading) due to the size of the TabsD bicubically interpolated dataset
bicubic_ds_tabsd = xr.open_dataset(
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_1971_2022_bicubic.nc",
    chunks={"time": 100} #Taking 100 days at a time
)
#Masking
masked_bicubic_tabsd = bicubic_ds_tabsd * mask_tabsd

masked_bicubic_tabsd.to_netcdf(
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_1km_bicubic_Swiss_features_masked.nc",
    engine="netcdf4",
    compute=True
)

In [4]:

precipitation_ds = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_1km_bicubic_Swiss_features_masked.nc")["pr"]

In [5]:
temperature_ds= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_1km_bicubic_Swiss_features_masked.nc")["tas"]

In [6]:
precipitation_ds.shape

(18993, 265, 370)

In [7]:
temperature_ds.shape

(18993, 265, 370)

In [8]:
rhires_d= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/RhiresD_1971_2022.nc")["RhiresD"]
rhires_d.shape

(18993, 265, 370)

In [9]:
tabs_d= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/TabsD_1971_2022.nc")["TabsD"]
tabs_d.shape

(18993, 265, 370)

#Standardising the dataset depending on the variable for feeding into the network

In [4]:
from Standardise import standardise

In [5]:
input_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/TabsD_1971_2022.nc'
output_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_scaled_1971_2022.nc'
var = 'TabsD'

standardise(input_path, output_path, var)


/users/sasthana/.local/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Normalised/min max scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/TabsD_scaled_1971_2022.nc


In [6]:
input_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/raw/RhiresD_1971_2022.nc'
output_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_scaled_1971_2022.nc'
var = 'RhiresD'

standardise(input_path, output_path, var)

Normalised/min max scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/RhiresD_scaled_1971_2022.nc
